# The Battle of Neighborhoods
## Toronto neighborhood - Vegan restaurants

**Author**: Hércules Sant'Ana da Silva José

_Report created in July/2020_

## Summary

**Introduction Section:**
- Introduction where we discuss the business problem about vegan restaurants in Toronto and who would be interested in this project.

**Data Section:**
- Data where we describe the data that will be used to solve the problem and the source of the data.

**Methodology section:**
- Methodology section which represents the main component of this report where we discuss and describe any exploratory data analysis that we did, any inferential statistical testing that we performed, if any, and what machine learnings were used and why.

**Results section:**
- Results section where we discuss the results.

**Discussion section:**
- Discussion section where we discuss any observations you noted and any recommendations to our audience based on the results.

**Conclusion section:**
- Conclusion section where we conclude the report.

## 1. Introduction

### 1.1. Scenario

Recent studies suggest that about 8% of the world's population is vegan. Few people around the world are committed to living a plant-based lifestyle. If you are interested in eating more cleanly, do not want to explore animals or simply enjoy how you feel as a vegan, it can sometimes be difficult to find great vegan restaurants in the world.

However, this is starting to change. You will find that the main cities in the world are slowly but surely developing strong vegan cultures, from coffee shops and cooperatives to some of the best vegan restaurants in the world. In short, you now have more options than simply ordering the only salad on the menu.

Toronto is one of the most ethnic and culturally diverse cities in the world. Located on the shores of Lake Ontario, it has people from all over the world from different cultures and ethnicities resulting in a lot of delicious delights. No matter what type of cuisine you like, you will surely find great restaurants that serve delicious food for you. Now, if you're a vegan, you may be wondering, all of this is good, but what if I'm looking for wonderful vegan cuisine to enjoy in Toronto? Will I be able to find something good? Will I as a student find good vegan restaurants near universities? Which neighborhoods can a company that specializes in vegan products promote its products? 

### 1.2. Business problem

Identify existing vegan restaurants in Toronto close to the city's universities, and identify neighborhoods where vegan companies can partner with restaurants to promote their products.

### 1.3. Audience

Vegan audience that comes to Toronto to study and companies interested in selling and promoting vegan products.

## 2. Data

### 2.1. Data description

For this analysis, we will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, the neighborhood data is not readily available on the internet. So, we will use a Wikipedia page that contains all the information we need to explore and cluster the neighborhoods in Toronto. The wikipedia page is available in https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

We will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format that we can do our analysis.

We will use the Foursquare API to identify the existing vegan restaurants in Toronto and existing restaurants near universities with a distance up to 3000 meters. We will use a list of boroughs and neighborhoods of Toronto with their coordinates (latitude and longitude) to obtain the venues of each boroughs and neighborhoods of Toronto. The coordinates dataset is available in http://cocl.us/Geospatial_data.

### 2.2. Data cleaning

The data cleaning process consisted of obtaining postal codes for the city of Toronto from Wikipedia page. All borough and neighborhood with value "Not assigned" were removed from the dataframe.

In [1]:
import pandas as pd

# Create dataframe by using pandas and show first 10 lines
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df[0].head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [2]:
# We've assigned index 0 to a new Pandas dataframe
df_toronto = df[0]

# Drop cells with a borough that is Not assigned
index_NA = df_toronto[(df_toronto['Borough']=='Not assigned')].index
df_toronto.drop(index_NA, inplace=True)
df_toronto.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto.loc[df_toronto.Neighbourhood=='Not assigned','Neighbourhood'] = df_toronto.Neighbourhood
df_toronto.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df_toronto.shape

(103, 3)

Then we obtain the geographic data of the Toronto neighborhoods. We merged the two dataframe, creating a new one with all neighborhoods and their respective coordinates (latitude and longitude).

In [5]:
# We used Pandas to read csv file that has the geographical coordinates of each postal code to a dataframe
df_coordinators = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinators.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [6]:
# We merged the df_coordinators into df_toronto.
df_toronto = pd.merge(left=df_toronto,right=df_coordinators,on='Postal Code')
df_toronto.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [7]:
# Check numbers of Boroughs and Neighborhoods in the df_toronto
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


We verify the geographic data collected by plotting the neighborhoods on the map.

In [8]:
import folium

# Toronto, Ontario
latitude = 43.6529
longitude = -79.3849

#Create map of Toronto using Latitude and Longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)

#add markers to map for neighborhoods
for lat,lng, borough, neighborhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

KeyError: 'Neighborhood'

### 2.3. Checking Foursquare API data

We invoke the Foursquare API to check the returned data and verify how many venues exists around the University of Toronto at a distance of up to 1000 meters.

In [9]:
# University of Toronto, Toronto, Ontario
neighborhood_latitude = 43.72149
neighborhood_longitude = -79.37881

In [10]:
# Foursquare credencials
CLIENT_ID = 'ERPDIEJOTYQ15D5OEZPG2BNWRX2XANXE5FS3A5XYM5ESFC4L' # your Foursquare ID
CLIENT_SECRET = 'VTEOE3I0I5SPVKSYMAURWWRCPBVR1GWBADW4H5KOX4JU4044' # your Foursquare Secret
VERSION = '20180604'

# URL
LIMIT=100
radius=1000
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT
)

In [11]:
import requests # library to handle requests

results = requests.get(url).json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

I've cleaned the returned json and I've structured it into a pandas dataframe.

In [13]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# First 10 rows
nearby_venues.head(10)

,name,categories,lat,lng
0,Sherwood Park,Park,43.716551,-79.387776
1,Whole Foods Market,Grocery Store,43.714160,-79.377966
2,Second Cup,Coffee Shop,43.721382,-79.376428
3,Dogs Off-Leash Area,Dog Run,43.716589,-79.384246
4,Rollian Sushi,Japanese Restaurant,43.712637,-79.376914
5,Tim Hortons,Coffee Shop,43.727324,-79.379563
6,Shoppers Drug Mart,Pharmacy,43.714158,-79.378098
7,Bistro On the Go: Sunnybrook,Food Court,43.721630,-79.376425
8,Druxy's,Deli / Bodega,43.720452,-79.377939
9,Glendon Forest,Trail,43.727226,-79.378413


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


## 3. Methodology

We get all the venues returned by the Foursquare API and check if there is a specific venue category for vegan restaurants.

We have created a function to facilitate the search for venues from the geographic data of the respective neighborhoods. We limited the search for venues to a distance of up to 500 meters.

In [15]:
# Function that return the nearby venues
# Distance: 500 meters
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                   'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
df_toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'],
                                  )
df_toronto_venues.head(10)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


For performance reasons, the data collected from Foursquare API was saved in a CSV file.

In [17]:
# Saving the results
df_toronto_venues.to_csv('toronto_venues.csv', index=False)
print('Data saved!')

Data saved!


In [18]:
import pandas as pd

# Loading the saved results
df_toronto_venues = pd.read_csv('toronto_venues.csv')
df_toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [19]:
print('Unique categories: {}.'.format(len(df_toronto_venues['Venue Category'].unique())))

Unique categories: 271.


In [20]:
df_toronto_venues['Venue Category'].unique()

array(['Park', 'Food & Drink Shop', 'Hockey Arena',
       'Portuguese Restaurant', 'Coffee Shop', 'French Restaurant',
       'Pizza Place', 'Bakery', 'Distribution Center', 'Spa',
       'Restaurant', 'Pub', 'Breakfast Spot', 'Gym / Fitness Center',
       'Historic Site', 'Farmers Market', 'Dessert Shop',
       'Chocolate Shop', 'Performing Arts Venue', 'Mexican Restaurant',
       'Café', 'Yoga Studio', 'Theater', 'Event Space',
       'Asian Restaurant', 'Shoe Store', 'Ice Cream Shop',
       'Electronics Store', 'Art Gallery', 'Bank', 'Beer Store', 'Hotel',
       'Wine Shop', 'Antique Shop', 'Boutique', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Clothing Store', 'Accessories Store',
       'Miscellaneous Shop', 'Creperie', 'Sushi Restaurant',
       'Arts & Crafts Store', 'Burrito Place', 'Beer Bar', 'Hobby Shop',
       'Diner', 'Fried Chicken Joint', 'Smoothie Shop', 'Sandwich Place',
       'Gym', 'College Auditorium', 'Bar', 'College Cafeteria',
       'Gene

## 4. Results

Exploring our dataset, we identified a specific category for vegan restaurants. 

In [21]:
# Verify that it exists a venue or venue category for 'vegan'
for i in df_toronto_venues['Venue Category'].unique():
    if 'Vegan' in i:
        print(i)
        break

Vegetarian / Vegan Restaurant


When listing all venues in the category, we found that there are only 19 venues registered in this category.

In [22]:
df_vegan_restaurants = df_toronto_venues[df_toronto_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
print('Vegan/Vegeratian restaurants in Toronto: {}.'.format(df_vegan_restaurants.shape[0]))

Vegan/Vegeratian restaurants in Toronto: 18.


In contrast, when we looked for 'restaurant', we found 46 venues categories.

In [23]:
# Counting all venues categories contain 'restaurant' except vegan restaurant.
i = 0
for item in df_toronto_venues['Venue Category'].unique():
    if 'Restaurant' in item and not 'Vegan' in item:
        i+=1
print('Quantity for restaurant category: ', i)

Quantity for restaurant category:  47


## 5. Discussion

We plot the vegan restaurants on the map together with the other restaurants categories.

In [27]:
df_all_restaurants = df_toronto_venues[df_toronto_venues['Venue Category'].str.contains('Restaurant')]

import folium

# Toronto, Ontario
latitude = 43.6529
longitude = -79.3849

#Create map of Toronto using Latitude and Longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=13)

#add markers to map for neighborhoods
for lat, lng, category, neighborhood in zip(df_all_restaurants['Venue Latitude'], df_all_restaurants['Venue Longitude'], df_all_restaurants['Venue Category'], df_all_restaurants['Neighborhood']):
    label = '{}, {}'.format(neighborhood, category)
    label = folium.Popup(label, parse_html=True)
    is_vegan = 'Vegan' in category
    folium.CircleMarker(
        [lat,lng],
        radius=3,
        popup=label,
        color='green' if is_vegan else 'red',
        fill=True,
        fill_color='green' if is_vegan else 'red',
        fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

We observed on the map that most vegan restaurants are present in places where other categories of restaurants already exist.

Students from the University of Toronto have a vegan restaurant option inside the campus, and a few options close to the university.

However, when comparing the number of existing restaurants with the number of vegan / vegetarian restaurants, we still have few options to serve an audience that continues to grow.

## 6. Conclusion

This report was intended to indicate which locations in Toronto have a vegan restaurant, and also to show potential locations for companies specializing in vegan products to partner with other restaurants to sell their products.

The data showed that there are nearby dining options for students at the University of Toronto, but overall the number of vegan restaurants is still small compared to the total number of existing restaurants.

## 7. References

https://bigseventravel.com/2020/02/best-vegan-restaurants-in-the-world-2020/

https://wtvox.com/sustainable-living/2019-the-world-of-vegan-but-how-many-vegans-are-in-the-world/

https://www.dailyhawker.ca/vegan-restaurants-in-toronto/

https://coursera.org
